In [ ]:
import numpy as np
import igl
import meshplot as mp
import scipy as sp
from scipy.spatial.transform import Rotation
from scipy.sparse.linalg import spsolve
from sksparse.cholmod import cholesky
import ipywidgets as iw
import time


H: set of handle vertices \
R: R remaining vertices not in H \
S: input surface (H + R = S)

In [ ]:
v, f = igl.read_triangle_mesh('data/hand.off')
labels = np.load('data/hand.label.npy').astype(int)
#v, f = igl.read_triangle_mesh('data/woody-hi.off')
#labels = np.load('data/woody-hi.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()

In [ ]:
handle_vertex_positions = v.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6))
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
pos_f.deformer = lambda x:x

In [ ]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

In [ ]:
def pre_factor_system(v, f, labels):
    Lw = igl.cotmatrix(v, f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())

    A = Lw@minv@Lw

    Aff = A[labels == 0][:, labels == 0]
    Afc = A[labels == 0][:, labels > 0]

    factor = cholesky(Aff)

    return factor, Afc
    #x = factor.solve_A(b)


In [ ]:
def smooth_deformer(target_pos):
    vc = target_pos[labels > 0]
    b = -Afc@vc
    x = factor(b)

    B = np.zeros(target_pos.shape)
    B[labels == 0] = x[:np.sum(labels == 0)]
    B[labels > 0] = target_pos[labels > 0]

    return B


In [ ]:
# step 3: Transferring high-frequency details to the deformed surface
def compute_high_freq_details(B, S, f):
    B_vertex_norms = igl.per_vertex_normals(B, f)
    neighbor_indices = igl.adjacency_list(f)

    x = np.zeros((B.shape[0], 3))
    y = np.zeros((B.shape[0], 3))
    d = np.zeros((B.shape[0], 3))
    d_x = np.zeros((B.shape[0]))
    d_y = np.zeros((B.shape[0]))
    d_n = np.zeros((B.shape[0]))
    neighbor_selections = np.zeros((B.shape[0]), dtype=int)
    for i in range(B.shape[0]):
        # build d
        d[i] = S[i] - B[i]

        # project all neighboring vertices of vi to the tangent plane (perpendicular to ni)
        #tang_planes = np.zeros((len(neighbor_indices[i]), 3))
        x_max = np.zeros((1, 3))
        for j in neighbor_indices[i]:
            tang_planes = B[j] - np.dot(B[j], B_vertex_norms[i]) * B_vertex_norms[i]
            if np.linalg.norm(tang_planes) > np.linalg.norm(x_max):
                x_max = tang_planes
                neighbor_selections[i] = j
        # find the neighbor j* for which projected edge (i, j) is longest. Normalize this edge vector and call it xi
        #x[i] = np.linalg.norm(tang_planes[np.argmax(np.linalg.norm(tang_planes, axis=1))])
        x[i] = x_max
        # construct yi using the cross product, completing orthonormal frame (xi, yi, ni)
        y[i] = np.cross(B_vertex_norms[i], x[i])

        # decompose the displacement vectors in the frame's basis: di = di_x xi + di_y yi + di_n ni
        # The bases is orthonormal, so you can do this just with inner products
        d_x[i] = np.dot(d[i], x[i])
        d_y[i] = np.dot(d[i], y[i])
        d_n[i] = np.dot(d[i], B_vertex_norms[i])

    return d_x, d_y, d_n, neighbor_selections

In [ ]:
def apply_high_freq_details(B_prime, d_x, d_y, d_n, neighbor_selections, f):
    B_prime_vertex_norms = igl.per_vertex_normals(B, f)
    x_prime = np.zeros((B_prime.shape[0], 3))
    y_prime = np.zeros((B_prime.shape[0], 3))
    d_prime = np.zeros((B_prime.shape[0], 3))
    for i in range(B_prime.shape[0]):
        j = neighbor_selections[i]
        x_prime[i] = B_prime[j] - np.dot(B_prime[j], B_prime_vertex_norms[i]) * B_prime_vertex_norms[i]
        y_prime[i] = np.cross(B_prime_vertex_norms[i], x_prime[i])
        d_prime[i] = d_x[i] * x_prime[i] + d_y[i] * y_prime[i] + d_n[i] * B_prime_vertex_norms[i]

    return B_prime + d_prime
    #mp.plot(S_prime, f)

In [ ]:
def position_deformer(target_pos):
    B = smooth_deformer(target_pos)
    return apply_high_freq_details(B, d_x, d_y, d_n, neighbor_selections, f)

    

In [ ]:
''' (Optional) Register this function to perform interactive deformation
pos_f.deformer = position_deformer
'''
pos_f.deformer = position_deformer

In [ ]:
mp.plot(v, f, c=labels)

In [ ]:
factor, Afc = pre_factor_system(v, f, labels)
B = smooth_deformer(v)
d_x, d_y, d_n, neighbor_selections = compute_high_freq_details(B, v, f)
mp.plot(B, f, c=labels)

In [ ]:
p = mp.plot(handle_vertex_positions, f, c=labels)
iw.interact(pos_f,
            **widgets_wrapper())

In [ ]:
B_prime = smooth_deformer(handle_vertex_positions)
mp.plot(B_prime, f, c=labels)

In [ ]:
S_prime = apply_high_freq_details(B_prime, d_x, d_y, d_n, neighbor_selections, f)
mp.plot(S_prime, f, c=labels)

In [ ]:
v, f = igl.read_triangle_mesh('data/woody-hi.off')
labels = np.load('data/woody-hi.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()

handle_vertex_positions = v.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6))

In [ ]:
mp.plot(v, f, c=labels)

In [ ]:
factor, Afc = pre_factor_system(v, f, labels)
B = smooth_deformer(v)
d_x, d_y, d_n, neighbor_selections = compute_high_freq_details(B, v, f)
mp.plot(B, f, c=labels)

In [ ]:
p = mp.plot(handle_vertex_positions, f, c=labels)
iw.interact(pos_f,
            **widgets_wrapper())

In [ ]:
B_prime = smooth_deformer(handle_vertex_positions)
mp.plot(B_prime, f, c=labels)

In [ ]:
S_prime = apply_high_freq_details(B_prime, d_x, d_y, d_n, neighbor_selections, f)
mp.plot(S_prime, f, c=labels)

In [ ]:
v, f = igl.read_triangle_mesh('')
labels = np.load('').astype(int)
v -= v.min(axis=0)
v /= v.max()

handle_vertex_positions = v.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6))

In [ ]:
v, f = igl.read_triangle_mesh('')
labels = np.load('').astype(int)
v -= v.min(axis=0)
v /= v.max()

handle_vertex_positions = v.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6))

old code below